## Wrangling the raw data in prep for Network Graph ##
In the Network Graph, want to show which Parent Entity has the most influence and in which platform. Only 1 value column is needed, so we need to merge all Subscriber Number into 1 column and to have another column to specify which platform those figures were from.

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
df1 = pd.read_excel("CANIS_PRC_state_media_on_social_media_platforms-2023-11-03.xlsx")
# drop Chinese columns
df1.drop(columns=["Name (Chinese)","Entity owner (Chinese)","Parent entity (Chinese)"], inplace=True)

In [3]:
columns_to_wrangle = ['X (Twitter) Follower #',
                      'Facebook Follower #',
                      'Instagram Follower #',
                      'Threads Follower #',
                      'YouTube Subscriber #',
                      'TikTok Subscriber #']

In [4]:
df1.head()

,Name (English),Region of Focus,Language,Entity owner (English),Parent entity (English),X (Twitter) handle,X (Twitter) URL,X (Twitter) Follower #,Facebook page,Facebook URL,Facebook Follower #,Instragram page,Instagram URL,Instagram Follower #,Threads account,Threads URL,Threads Follower #,YouTube account,YouTube URL,YouTube Subscriber #,TikTok account,TikTok URL,TikTok Subscriber #
0,Yang Xinmeng (Abby Yang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,_bubblyabby_,https://twitter.com/_bubblyabby_,1678.0,itsAbby-103043374799622,https://www.facebook.com/itsAbby-103043374799622,1387432.0,_bubblyabby_,https://www.instagram.com/_bubblyabby_/,9507.0,_bubblyabby_,https://www.threads.net/@_bubblyabby_,197.0,itsAbby,https://www.youtube.com/itsAbby,4680.0,_bubblyabby_,https://www.tiktok.com/@_bubblyabby_,660.0
1,CGTN Culture Express,Anglosphere,English,China Media Group (CMG),Central Publicity Department,_cultureexpress,https://twitter.com/_cultureexpress,2488.0,NaN,NaN,NaN,_cultureexpress/,https://www.instagram.com/_cultureexpress/,635.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,All-China Students' Federation,China,Chinese,All-China Students' Federation,Central Committee of the Communist Youth League,ACSF1919,https://twitter.com/ACSF1919,5136.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chen Zhong (Dechinghutay / Ghoti),Afghanistan,Pashto,China Media Group (CMG),Central Publicity Department,afghanchina,https://twitter.com/afghanchina,1800.0,dechinghutay,https://www.facebook.com/dechinghutay/,1600000.0,NaN,NaN,NaN,NaN,NaN,NaN,dechinghutay,https://www.youtube.com/channel/UCXl9X2fi65wKf...,695.0,dechinghutay,https://www.tiktok.com/@dechinghutay,326100.0
4,Yang Sheng,Anglosphere,English,People's Daily Press,Central Committee of the Chinese Communist Party,AlexYsalex17,https://twitter.com/AlexYsalex17,2821.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Drop account name or handle and links for each platform. ###
Only wanted the subscriber numbers for network graph

In [5]:
columns_to_drop = [5,6,8,9,11,12,14,15,17,18,20,21]
df1.drop(df1.columns[columns_to_drop], axis=1, inplace=True)
df1.head()

,Name (English),Region of Focus,Language,Entity owner (English),Parent entity (English),X (Twitter) Follower #,Facebook Follower #,Instagram Follower #,Threads Follower #,YouTube Subscriber #,TikTok Subscriber #
0,Yang Xinmeng (Abby Yang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,1678.0,1387432.0,9507.0,197.0,4680.0,660.0
1,CGTN Culture Express,Anglosphere,English,China Media Group (CMG),Central Publicity Department,2488.0,NaN,635.0,NaN,NaN,NaN
2,All-China Students' Federation,China,Chinese,All-China Students' Federation,Central Committee of the Communist Youth League,5136.0,NaN,NaN,NaN,NaN,NaN
3,Chen Zhong (Dechinghutay / Ghoti),Afghanistan,Pashto,China Media Group (CMG),Central Publicity Department,1800.0,1600000.0,NaN,NaN,695.0,326100.0
4,Yang Sheng,Anglosphere,English,People's Daily Press,Central Committee of the Chinese Communist Party,2821.0,NaN,NaN,NaN,NaN,NaN


### Create two new columns: Platform and Followers ###
- Rearrange table and sort into rows with most platforms (computing a simple statistics along the way)
- Remove any rows that are null or without any followers on any of the platforms
- Wrangle and Melt the Follower or Subscriber columns into one column

In [6]:
columns_to_count = ['X (Twitter) Follower #',
                    'Facebook Follower #',
                    'Instagram Follower #',
                    'Threads Follower #',
                    'YouTube Subscriber #',
                    'TikTok Subscriber #']
df1['NumAccount'] = df1[columns_to_count].count(axis=1)
df1 = df1.sort_values(by='NumAccount', ascending=False).reset_index(drop=True)
#df_sorted.drop(columns='non_null_count', inplace=True)
df1 = df1[df1['NumAccount'] > 0]
NumAccount_Count = []
for i in range(6,0,-1):
    NumAccount_Count.append(len(df1[df1["NumAccount"] == i]))
NumAccount_Count

[10, 24, 42, 82, 157, 442]

In [7]:
len(df1)

757

In [8]:
list(df1.columns)

['Name (English)',
 'Region of Focus',
 'Language',
 'Entity owner (English)',
 'Parent entity (English)',
 'X (Twitter) Follower #',
 'Facebook Follower #',
 'Instagram Follower #',
 'Threads Follower #',
 'YouTube Subscriber #',
 'TikTok Subscriber #',
 'NumAccount']

In [9]:
df1.head(10)

,Name (English),Region of Focus,Language,Entity owner (English),Parent entity (English),X (Twitter) Follower #,Facebook Follower #,Instagram Follower #,Threads Follower #,YouTube Subscriber #,TikTok Subscriber #,NumAccount
0,Yang Xinmeng (Abby Yang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,1678.0,1387432.0,9507.0,197.0,4680.0,660.0,6
1,China Daily,Anglosphere,English,China Daily Press,Central Publicity Department,4100000.0,104613175.0,1300000.0,40200.0,337000.0,1300000.0,6
2,Where Zhengzhou,Anglosphere,English,Zhengzhou Media Group,Zhengzhou Municipal Party Committee,14900.0,10000.0,26400.0,40.0,1470.0,5961.0,6
3,People's Daily,China,Chinese,People's Daily Press,Central Committee of the Chinese Communist Party,6600000.0,85867335.0,1300000.0,28200.0,410000.0,2600000.0,6
4,Miao Xiaojuan,Anglosphere,English,Xinhua News Agency,State Council,6577.0,1800000.0,2445.0,85.0,1030.0,3.0,6
5,Li Jingjing,Anglosphere,English,China Media Group (CMG),Central Publicity Department,30000.0,2900000.0,23900.0,950.0,58300.0,17200.0,6
6,Zang Shijie (Jessica Zang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,3083.0,1200000.0,8413.0,173.0,967.0,16800.0,6
7,iPanda,Anglosphere,English,China Media Group (CMG),Central Publicity Department,159500.0,24000000.0,2400000.0,121000.0,2180000.0,2500000.0,6
8,iChongqing,Chongqing,English,Chongqing International Communication Center,Chongqing Municipal Committee,189300.0,7000000.0,11500.0,380.0,26900.0,632.0,6
9,Li Yimei (Vica Li),Anglosphere,English,China Media Group (CMG),Central Publicity Department,1707.0,1300000.0,115000.0,6669.0,7290.0,8885.0,6


In [10]:
columns_to_keep = ['Name (English)', 'Region of Focus', 'Language', 
                   'Entity owner (English)','Parent entity (English)',]
df1_melted = df1.melt(id_vars=columns_to_keep, value_vars=columns_to_count,
                     var_name='Platform', value_name='Followers')

In [11]:
df1_melted[df1_melted["Platform"]=="Instagram Follower #"].head(20)

,Name (English),Region of Focus,Language,Entity owner (English),Parent entity (English),Platform,Followers
1514,Yang Xinmeng (Abby Yang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,Instagram Follower #,9507.0
1515,China Daily,Anglosphere,English,China Daily Press,Central Publicity Department,Instagram Follower #,1300000.0
1516,Where Zhengzhou,Anglosphere,English,Zhengzhou Media Group,Zhengzhou Municipal Party Committee,Instagram Follower #,26400.0
1517,People's Daily,China,Chinese,People's Daily Press,Central Committee of the Chinese Communist Party,Instagram Follower #,1300000.0
1518,Miao Xiaojuan,Anglosphere,English,Xinhua News Agency,State Council,Instagram Follower #,2445.0
1519,Li Jingjing,Anglosphere,English,China Media Group (CMG),Central Publicity Department,Instagram Follower #,23900.0
1520,Zang Shijie (Jessica Zang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,Instagram Follower #,8413.0
1521,iPanda,Anglosphere,English,China Media Group (CMG),Central Publicity Department,Instagram Follower #,2400000.0
1522,iChongqing,Chongqing,English,Chongqing International Communication Center,Chongqing Municipal Committee,Instagram Follower #,11500.0
1523,Li Yimei (Vica Li),Anglosphere,English,China Media Group (CMG),Central Publicity Department,Instagram Follower #,115000.0


In [12]:
# Get rid of the " Follower #" and " Subscriber #" in Platform column
df1_melted['Platform'] = df1_melted['Platform'].replace(" Follower #", "", regex=True)
df1_melted['Platform'] = df1_melted['Platform'].replace(" Subscriber #", "", regex=True)

In [13]:
df1_melted[df1_melted["Platform"].str.contains("Twitter")].head(1)

,Name (English),Region of Focus,Language,Entity owner (English),Parent entity (English),Platform,Followers
0,Yang Xinmeng (Abby Yang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,X (Twitter),1678.0


In [14]:
df1_melted[df1_melted["Platform"].str.contains("Facebook")].head(1)

,Name (English),Region of Focus,Language,Entity owner (English),Parent entity (English),Platform,Followers
757,Yang Xinmeng (Abby Yang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,Facebook,1387432.0


In [15]:
df1_melted[df1_melted["Platform"].str.contains("Instagram")].head(1)

,Name (English),Region of Focus,Language,Entity owner (English),Parent entity (English),Platform,Followers
1514,Yang Xinmeng (Abby Yang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,Instagram,9507.0


In [16]:
df1_melted[df1_melted["Platform"].str.contains("Threads")].head(1)

,Name (English),Region of Focus,Language,Entity owner (English),Parent entity (English),Platform,Followers
2271,Yang Xinmeng (Abby Yang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,Threads,197.0


In [17]:
df1_melted[df1_melted["Platform"].str.contains("YouTube")].head(1)

,Name (English),Region of Focus,Language,Entity owner (English),Parent entity (English),Platform,Followers
3028,Yang Xinmeng (Abby Yang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,YouTube,4680.0


In [18]:
df1_melted[df1_melted["Platform"].str.contains("TikTok")].head(1)

,Name (English),Region of Focus,Language,Entity owner (English),Parent entity (English),Platform,Followers
3785,Yang Xinmeng (Abby Yang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,TikTok,660.0


In [19]:
# drop null in Followers columns before exporting
df1_filtered = df1_melted.dropna(subset=["Followers"])

In [ ]:
df1_filtered.to_csv("dataset_for_NetworkGraph.csv", index=False)